In [ ]:
from huggingface_hub import hf_hub_download
file = hf_hub_download('alexedw/dense-train-masked-between-tokens', 'model_state.pt', revision='1500')


In [ ]:
file

In [ ]:
import sys
sys.path.append('..')
from transformers import GPTNeoXTokenizerFast
from core.model import GPT, GPTConfig, Tokenizer, THOUGHT_TOKEN_ID
import torch
tokenizer = Tokenizer()

model = GPT(GPTConfig.from_pretrained('EleutherAI/pythia-410m'))
state_dict = torch.load(file, map_location=torch.device('cpu'))

unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

model.load_state_dict(state_dict)
state_dict = None

In [ ]:
text_to_encode="""
MIT License

Copyright (c) 2023 Zela Labs

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
"""

tokens = tokenizer.encode(f'{text_to_encode}<|dense|><|dense|><|dense|>').unsqueeze(0)

logits, dense, loss = model(tokens, model.create_dense_inputs(tokens))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))

In [ ]:
tokens_new = model.generate(torch.tensor([[50277, 50277, 50277]]), dense_input=dense[:, -3:, :], max_new_tokens=25, temperature=0.0001)
tokens_new, tokenizer.decode(tokens_new[0])

In [ ]:
tokens_new = model.generate(torch.tensor([[50277, 50277, 50277]]), max_new_tokens=15, temperature=0.0001)
tokens_new, tokenizer.decode(tokens_new[0])

# Test number 2... encoding a memory?

In [338]:
text_to_encode="""My name is Alex. Alex Alex Alex loves the colour blue blue blue"""
tokens = tokenizer.encode(f'{text_to_encode}<|dense|><|dense|><|dense|>').unsqueeze(0)
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))

name_one_dense = dense[:, -3:, :] * 2

text_to_encode="""My name is Sarah. Sarah Sarah Sarah loves the colour green green green"""
tokens = tokenizer.encode(f'{text_to_encode}<|dense|><|dense|><|dense|>').unsqueeze(0)
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))
logits, dense, loss = model(tokens, model.create_dense_inputs(tokens, dense))

name_two_dense = dense[:, -3:, :] * 2

question_one_tokens = tokenizer.encode("Hi there, I'd like to introduce myself, my name is<|dense|><|dense|><|dense|>").unsqueeze(0)
question_two_tokens = tokenizer.encode("Hi there, my favourite colour is<|dense|><|dense|><|dense|>").unsqueeze(0)

In [339]:
dense_inputs = model.create_dense_inputs(question_one_tokens)
dense_inputs[:, -3:, :] = name_one_dense
tokens_new = model.generate(question_one_tokens, dense_input=dense_inputs, max_new_tokens=25, temperature=0.0001)
print(tokenizer.decode(tokens_new[0]))

Hi there, I'd like to introduce myself, my name is<|dense|><|dense|><|dense|>. I'm a student at the University of the Arts in London. I'm a student in the English department. I'm


In [340]:
dense_inputs = model.create_dense_inputs(question_one_tokens)
dense_inputs[:, -3:, :] = name_two_dense
tokens_new = model.generate(question_one_tokens, dense_input=dense_inputs, max_new_tokens=25, temperature=0.0001)
print(tokenizer.decode(tokens_new[0]))

Hi there, I'd like to introduce myself, my name is<|dense|><|dense|><|dense|> green. I'm a student at the University of the Arts in London. I'm a student in the English department and I


In [341]:
dense_inputs = model.create_dense_inputs(question_two_tokens)
dense_inputs[:, -3:, :] = name_one_dense
tokens_new = model.generate(question_two_tokens, dense_input=dense_inputs, max_new_tokens=25, temperature=0.0001)
print(tokenizer.decode(tokens_new[0]))

Hi there, my favourite colour is<|dense|><|dense|><|dense|>. I love it so much. I have a few of my own and I love them. I have a few of my


In [342]:
dense_inputs = model.create_dense_inputs(question_two_tokens)
dense_inputs[:, -3:, :] = name_two_dense
tokens_new = model.generate(question_two_tokens, dense_input=dense_inputs, max_new_tokens=25, temperature=0.0001)
print(tokenizer.decode(tokens_new[0]))

Hi there, my favourite colour is<|dense|><|dense|><|dense|> green. I love the colour of the sky and the sky is always green. I love the colour of the sky and the
